In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [2]:
gc = GraphCreator("Decision tree", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 257 


['Goal', 'Algorithm', 'Operations research', 'Decision tree learning', 'Decision support system', 'Probability', 'Utility', 'Decision analysis', 'Causal model', 'Tree (graph theory)'] 


['Behavior tree (artificial intelligence, robotics and control)', 'Boosting (machine learning)', 'Decision cycle', 'Decision list', 'Decision table', 'Decision tree model', 'Design rationale', 'DRAKON', 'Markov chain', 'Random forest', 'Odds algorithm', 'Topological combinatorics', 'Truth table']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [3]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [4]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [5]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Hysteria Project 2,Hysteria Project,7
1,Hysteria Project,Hysteria Project 2,7
2,Mind map,Concept map,5
3,Deep learning,Glossary of artificial intelligence,5
4,Concept map,Mind map,5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [25]:
features_df = gc.get_features_df(rank=False)

In [26]:
features_df.sort_values("degree", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link
0,CiteSeerX,14457,0,14397,60,0.002668,2.132735e-01,0.085880,0.011531,1.0,3.0,0.001857,0
1,Logic,3493,0,2794,699,0.005084,9.310650e-02,0.021339,0.214227,2.0,1.0,0.003726,0
2,Algorithm,3381,0,2993,388,0.011272,1.654078e-01,0.016771,0.062508,1.0,2.0,0.008291,1
3,Index of philosophy articles (D–H),3034,0,13,3021,0.002162,3.354317e-04,0.000090,0.006069,3.0,1.0,0.005525,0
4,Time,2896,0,1970,926,0.003509,5.348922e-02,0.014041,0.329533,1.0,3.0,0.000468,0
5,Dungeon Master,2676,0,2342,334,0.000384,7.286231e-06,0.030140,0.177814,4.0,1.0,0.000000,0
6,Machine learning,2610,0,2081,529,0.023366,2.794621e-01,0.012022,0.149898,2.0,1.0,0.017639,0
7,Game theory,2555,0,1795,760,0.013363,1.362012e-01,0.011308,0.304044,2.0,1.0,0.006663,0
8,Causality,2279,0,1280,999,0.007485,6.599033e-02,0.008751,0.291907,1.0,2.0,0.002768,0
9,Glossary of artificial intelligence,2022,0,319,1703,0.020329,1.875950e-01,0.002333,0.081928,2.0,1.0,0.027368,0


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [ ]:
gc.scale_features_df(scaler=MinMaxScaler, copy=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [ ]:
gc.rank_similarity()
gc.scale_features_df(scaler=MinMaxScaler, copy=False)
gc.features_df.sort_values(["similarity_rank"], ascending=False).reset_index().drop("index", axis=1)

In [27]:
gc.features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54506 entries, 0 to 54505
Data columns (total 13 columns):
node                                 54506 non-null object
degree                               54506 non-null int64
category_matches_with_source         54506 non-null int64
in_edges                             54506 non-null int64
out_edges                            54506 non-null int64
shared_neighbors_with_entry_score    54506 non-null float64
centrality                           54506 non-null float64
page_rank                            54506 non-null float64
adjusted_reciprocity                 54506 non-null float64
shortest_path_length_from_entry      54506 non-null float64
shortest_path_length_to_entry        54506 non-null float64
jaccard_similarity                   54506 non-null float64
primary_link                         54506 non-null int64
dtypes: float64(7), int64(5), object(1)
memory usage: 5.8+ MB


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [ ]:
evaluate_metrics(gc.features_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [ ]:
df = gc.features_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)
see_also_indeces = []
for node in gc.see_also_articles:
    article = df[df.node == node]
    
    see_also_indeces.append((node, article.index[0]))
    
see_also_indeces

In [ ]:
len(gc.graph.nodes)